Импорт библиотек

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import telebot
from telebot import types

Подключение к таблице для создания датафрейма

In [2]:
df_shedule = pd.read_excel('..\SheduleFESU.xlsx', header=0, usecols='A:J')
# df_shedule.loc[df_shedule["Profile"] == ' Социология', "Profile"] = "Социология" #df = df.replace(to_replace=r'.*', value=new_value, regex=True)
df_shedule.fillna(value='Окно', inplace=True)
display(df_shedule.isna().sum())


Courses       0
Profile       0
Group         0
Num_Day       0
Day           0
Num_Lesson    0
Subject       0
Room          0
Week          0
Logo          0
dtype: int64

In [3]:
group_names = df_shedule['Group'].unique()
len(group_names)
logo = "https://img.icons8.com/?size=100&id=EVWPoM8ohVl2&format=png&color=000000"

nm_lesson = df_shedule.loc[df_shedule['Num_Day'] == 2, 'Num_Lesson'].values
nm_lesson
subject = df_shedule.loc[(df_shedule['Num_Day'] == int(1)) & (df_shedule['Group'] == int(9409)), 'Subject'].values
subject

array(['Иностранный язык ', 'Культурология', 'Основы социологии', 'ИКТ'],
      dtype=object)

In [4]:
name_days = pd.DataFrame({
        "1": ["Понедельник"],
        "2": ["Вториник"],
        "3": ["Среда"],
        "4": ["Четверг"],
        "5": ["Пятница"],
        "6": ["Суббота"],
        "7": ["Воскресенье"]}) #, index=range(7))

name_days = name_days.stack().reset_index(drop=True)
name_days

0    Понедельник
1       Вториник
2          Среда
3        Четверг
4        Пятница
5        Суббота
6    Воскресенье
dtype: object

Проверка логики программы

In [5]:
def get_text_messages( message_group = input('Enter num group: '), message_day = input('Enter num day: ') ): #message_course = input('Enter num course: '),
    num_of_week_day = message_day.strip()
    # num_of_course = message_course.strip()
    num_of_group = message_group.strip()
    subject = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Subject'].values
    nm_lesson = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Num_Lesson'].values
    # print(f'{num_of_course} {num_of_group} {num_of_week_day}')
    
    if len(subject) > 0:
        i = 0
        for sub in subject:
            print(f"{df_shedule[df_shedule['Num_Day'] == int(num_of_week_day)]['Day'].values[0]} Пара # {nm_lesson[i]} по дисциплине: {sub}")
            i += 1

    else: 
        print(f'В {[name_days[int(num_of_week_day)- 1]][0]} у Вас пар нет')


get_text_messages()

В Понедельник у Вас пар нет


# Create bot Telegram

In [2]:
# 1)Create bot 2)Give token 3)Task like Neyro 4) Add pandas 5)Thin abot Logic 1010)task server Heroku -> RSU
load_dotenv()
TOKEN_BOT_RSU = os.getenv('TOKEN_BOT_RSU') 


In [ ]:
# next(iter(user_data.values()))
# first_key, first_value = next(iter(user_data.items()))
# print(f"Ключ: {first_key}, Значение: {first_value}")



Variant 3


Работает с датафреймом но кнопки не нравятся

In [ ]:
univercity_df = pd.DataFrame(columns= ['course', "group", "nm"], data=[[1,9402, 25], [1,9403, 20], [2,9302,15]])
univercity_df.loc[len(univercity_df)] = [2,9308,18]
# univercity_df[univercity_df["course"] == 2]["group"].values[0:][0]

bot = telebot.TeleBot(TOKEN_BOT_RSU)

user_state = {}  # Словарь для хранения состояния каждого пользователя

@bot.message_handler(commands=['start']) # Функция для обработки команды /start
def start_command(message):
    user_id = message.from_user.id
    user_state[user_id] = {}  # Инициализация состояния для нового пользователя
    main_menu(message)

# Основная клавиатура с выбором курса
def main_menu(message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    btn1 = types.KeyboardButton("Курс_1") # текст для передачи в Message след функции  message.text
    btn2 = types.KeyboardButton("Курс_2")
    keyboard.add(btn1, btn2)
    bot.send_message(message.chat.id, "Выберите курс:", reply_markup=keyboard)

# Подменю первой категории
def submenu_category1(message):
    nm_corse = message.text.split('_')[1]
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    btn11 = types.KeyboardButton(f'Группа_{univercity_df[univercity_df["course"] == int(nm_corse)]["group"].values[0:][0]}') # выводит кнопку с номером группы => передаем в след фнкц
    btn12 = types.KeyboardButton(f'Группа_{univercity_df[univercity_df["course"] == int(nm_corse)]["group"].values[0:][1]}')
    # list_btns = []
    keyboard.add(btn11, btn12)
    bot.send_message(message.chat.id, "Выберите группу:", reply_markup=keyboard)

# Подменю второй категории
def submenu_category2(message):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    btn21 = types.KeyboardButton("Опция 2.1")
    btn22 = types.KeyboardButton("Опция 2.2")
    keyboard.add(btn21, btn22)
    bot.send_message(message.chat.id, "Выберите опцию:", reply_markup=keyboard)

# Обработчик всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def process_message(message):
    user_id = message.from_user.id
    state = user_state.get(user_id, {})

    if not state:  # Если состояние пустое, значит, пользователь еще ничего не выбрал
        if message.text != "":
            state['coruse'] = message.text
            submenu_category1(message)
        # elif message.text == "Курс_2":
        #     state['coruse'] = "Курс_2"
        #     submenu_category1(message) # maybe 2 return
        else:
            bot.reply_to(message, "Пожалуйста, выберите курс.")
    else:
        # Состояние уже установлено, значит, пользователь выбирает опции внутри выбранной категории
        category = state['coruse']
        if message.text != "":
                state['group'] = message.text
                nm_grp = message.text.split('_')[1]
                bot.reply_to(message, f'''Ваш курс: {state['coruse']}, Ваша группа: {state['group']}! \n 
                            Численность группы {univercity_df[univercity_df['group'] == int(nm_grp)]['nm'].values[0:][0]}''')
        else:
                bot.reply_to(message, "Пожалуйста, выберите одну из опций.")

bot.polling()

В продакшн

In [ ]:
bot = telebot.TeleBot(TOKEN_BOT_RSU)
user_state = {}  # Словарь для хранения состояния каждого пользователя

@bot.message_handler(commands=['start'])
def start_command(message):
    user_id = message.from_user.id
    user_state[user_id] = {}  # инициализация состояния
    user_state[user_id]['name'] = message.from_user.first_name
    name = user_state[user_id]['name']
    show_main_menu(message.chat.id, name)

def show_main_menu(chat_id, name):
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    btn1 = types.InlineKeyboardButton("Курс_1", callback_data="course_1")
    btn2 = types.InlineKeyboardButton("Курс_2", callback_data="course_2")
    btn3 = types.InlineKeyboardButton("Курс_3", callback_data="course_3")
    btn4 = types.InlineKeyboardButton("Курс_4", callback_data="course_4")
    keyboard.add(btn1, btn2, btn3, btn4)
    bot.send_message(chat_id, f'''Привет! {name} выбери курс ''', reply_markup=keyboard)

def show_submenu_category1(chat_id, course_number):
    # получаем список групп для курса
    groups = univercity_df[univercity_df["course"] == int(course_number)]["group"].values # отбор курса добавить
    keyboard = types.InlineKeyboardMarkup(row_width=2)
    # btn_back = types.InlineKeyboardButton("🔙 Назад", callback_data="back_main")
    for grp in groups[:]:  # берем первые две группы а надо все!!!!
        btn = types.InlineKeyboardButton(f"Группа_{grp}", callback_data=f"group_{grp}")
        keyboard.add(btn)
    bot.send_message(chat_id, "Выберите группу:", reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: call.data.startswith(('course_', 'group_')))
def callback_query(call):
    user_id = call.from_user.id
    data = call.data  # например, "course_1" или "group_101"
    
    # если выбираем курс
    if data.startswith("course_"):
        course_number = data.split("_")[1]
        user_state[user_id]['course'] = f"Курс_{course_number}"
        # отправляем подменю групп
        show_submenu_category1(call.message.chat.id, course_number)
        bot.answer_callback_query(call.id)  # убираем "часики" на кнопке

    # если выбираем группу
    elif data.startswith("group_"):
        group_number = data.split("_")[1]
        user_state[user_id]['group'] = f"Группа_{group_number}"
        state = user_state[user_id]
        # достаём численность группы из dataframe
        nm = univercity_df[univercity_df['group'] == int(group_number)]['nm'].values[0]
        text = (f"Ваш курс: {state['course']}\n"
                f"Ваша группа: {state['group']}\n"
                f"Численность группы: {nm}")
        bot.send_message(call.message.chat.id, text)
        bot.answer_callback_query(call.id)


bot.polling()